##### Copyright 2023 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2023 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <!-- <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/bird_vocalization_classifier"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td> -->
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/bird_vocalization_classifier.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/bird_vocalization_classifier.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub で表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/bird_vocalization_classifier.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
  <td><a href="https://tfhub.dev/google/bird-vocalization-classifier/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF Hub モデルを参照</a></td>
</table>

# Google Bird Vocalization モデルの使用

Google Bird Vocalization は、グローバルな鳥類の埋め込みと分類モデルです。

このモデルは、入力として、32kHz でサンプリングされた 5 秒間の音声セグメントを期待します。

モデルは、音声の入力ウィンドウごとにロジットと埋め込みの両方を出力します。

このノートブックでは、モデルに音声を適切にフィードする方法と推論にロジットを使用する方法について学習します。


In [ ]:
!pip install -q "tensorflow_io==0.28.*"
!pip install -q librosa

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

import numpy as np
import librosa

import csv
import io

from IPython.display import Audio

TFHub からのモデルの読み込み

In [ ]:
model_handle = "https://tfhub.dev/google/bird-vocalization-classifier/1"
model = hub.load(model_handle)

モデルをトレーニングしたラベルを読み込みましょう。

labels ファイルは、assets フォルダの labe.cvs にあります。

In [ ]:
# Find the name of the class with the top score when mean-aggregated across frames.
def class_names_from_csv(class_map_csv_text):
  """Returns list of class names corresponding to score vector."""
  with open(labels_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    class_names = [mid for mid, desc in csv_reader]
    return class_names[1:]

labels_path = hub.resolve(model_handle) + "/assets/label.csv"
classes = class_names_from_csv(labels_path)
print(classes)

`frame_audio` 関数は [Chirp ライブラリ](https://github.com/google-research/chirp/blob/10c5faa325a3c3468fa6f18a736fc1aeb9bf8129/chirp/inference/interface.py#L128) バージョンを基盤としていますが、librosa の代わりに tf.signal を使用しています。

`ensure_sample_rate` は、モデルに使用される音声に、期待される 32kHzのサンプリングレートが使用されていることを確認する関数です。

In [ ]:
def frame_audio(
      audio_array: np.ndarray,
      window_size_s: float = 5.0,
      hop_size_s: float = 5.0,
      sample_rate = 32000,
  ) -> np.ndarray:
    """Helper function for framing audio for inference."""
    if window_size_s is None or window_size_s < 0:
      return audio_array[np.newaxis, :]
    frame_length = int(window_size_s * sample_rate)
    hop_length = int(hop_size_s * sample_rate)
    framed_audio = tf.signal.frame(audio_array, frame_length, hop_length, pad_end=True)
    return framed_audio

def ensure_sample_rate(waveform, original_sample_rate,
                       desired_sample_rate=32000):
  """Resample waveform if required."""
  if original_sample_rate != desired_sample_rate:
    waveform = tfio.audio.resample(waveform, original_sample_rate, desired_sample_rate)
  return desired_sample_rate, waveform

Wikipedia からファイルを読み込んでみましょう。

より正確には、[クロウタドリ](https://es.wikipedia.org/wiki/Turdus_merula)の音声です。

<p><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Common_Blackbird.jpg/1200px-Common_Blackbird.jpg" alt="Common Blackbird.jpg" class=""></p>
:-:
*By <a rel="nofollow" class="external text" href="http://photo-natur.de">Andreas Trepte</a> - <span class="int-own-work" lang="en">Own work</span>, <a href="https://creativecommons.org/licenses/by-sa/2.5" title="Creative Commons Attribution-Share Alike 2.5">CC BY-SA 2.5</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=16110223">Link</a>*

この音声はパブリックドメインライセンスの下、Oona Räisänen (Mysid) によって貢献されたものです。

In [ ]:
!curl -O  "https://upload.wikimedia.org/wikipedia/commons/7/7c/Turdus_merula_2.ogg"

In [ ]:
turdus_merula = "Turdus_merula_2.ogg"

audio, sample_rate = librosa.load(turdus_merula)

sample_rate, wav_data_turdus = ensure_sample_rate(audio, sample_rate)
Audio(wav_data_turdus, rate=sample_rate)

音声は 24 秒で、モデルは 5 秒を期待しています。

これは、`frame_audio` 関数を使って修正し、適切なフレームに音声を分割できます。

In [ ]:
fixed_tm = frame_audio(wav_data_turdus)
fixed_tm.shape

では、最初のフレームにモデルを適用してみましょう。

In [ ]:
logits, embeddings = model.infer_tf(fixed_tm[:1])

label.csv ファイルには、ebirds id が含まれています。Turdus Merula（クロウタドリツグミ）の ebird id は eurbla です。

In [ ]:
probabilities = tf.nn.softmax(logits)
argmax = np.argmax(probabilities)
print(f"The audio is from the class {classes[argmax]} (element:{argmax} in the label.csv file), with probability of {probabilities[0][argmax]}")

では、すべてのフレームにモデルを適用してみましょう。

*注意*: このコードも [Chirp ライブラリ](https://github.com/google-research/chirp/blob/d6ff5e7cee3865940f31697bf4b70176c1072572/chirp/inference/models.py#L174)に基づいています。

In [ ]:
all_logits, all_embeddings = model.infer_tf(fixed_tm[:1])
for window in fixed_tm[1:]:
  logits, embeddings = model.infer_tf(window[np.newaxis, :])
  all_logits = np.concatenate([all_logits, logits], axis=0)

all_logits.shape

In [ ]:
frame = 0
for frame_logits in all_logits:
  probabilities = tf.nn.softmax(frame_logits)
  argmax = np.argmax(probabilities)
  print(f"For frame {frame}, the audio is from the class {classes[argmax]} (element:{argmax} in the label.csv file), with probability of {probabilities[argmax]}")
  frame += 1